# Model Exploration
This objective of this project is to evaluates 3 approaches to accurately analyze real-world data: a naive approach, a non deep learning approach, and a neural network-based deep learning approach

In [2]:
# Imports
import os
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights
from dotenv import load_dotenv
from huggingface_hub import HfApi, login
import warnings

warnings.filterwarnings('ignore')

In [3]:
train = np.load('./data/processed/train.npz')
val = np.load('./data/processed/val.npz')
test = np.load('./data/processed/test.npz')

X_train, y_train = train['X'], train['y']
X_val, y_val = val['X'], val['y']
X_test, y_test = test['X'], test['y']

class_names = train['class_names']

### Naive Approach


In [ ]:
unique_classes, counts = np.unique(y_train, return_counts=True)
majority_class = unique_classes[np.argmax(counts)] # Get majority class

y_pred_test = np.full_like(y_test, majority_class) # Predict majority class for all

In [ ]:
accuracy = accuracy_score(y_test, y_pred_test)
precision = precision_score(y_test, y_pred_test, average='macro', zero_division=0)
recall = recall_score(y_test, y_pred_test, average='macro', zero_division=0)
f1 = f1_score(y_test, y_pred_test, average='macro', zero_division=0)

print('Accuracy:', round(accuracy, 4))
print('Precision:', round(precision, 4))
print('Recall:', round(recall, 4))
print('F1-score:', round(f1, 4))

Accuracy: 0.0029
Precision: 0.0
Recall: 0.0029
F1-score: 0.0


### Classical Machine Learning Approach


In [20]:
# Flatten images
X_train_flat = X_train.reshape(len(X_train), -1)
X_test_flat = X_test.reshape(len(X_test), -1)

In [ ]:
# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_flat)
X_test_scaled = scaler.transform(X_test_flat)

In [ ]:
pca = PCA(n_components=50, random_state=42) # PCA to reduce dimensions
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca  = pca.transform(X_test_scaled)

In [ ]:
model = LogisticRegression(multi_class='multinomial', max_iter=200, n_jobs=-1, random_state=42)
model.fit(X_train_pca, y_train)

c:\Users\maria\Documents\draw_to_speech\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,42
,solver,'lbfgs'
,max_iter,200
,multi_class,'multinomial'


In [ ]:
y_pred = model.predict(X_test_pca)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='macro', zero_division=0)
recall = recall_score(y_test, y_pred, average='macro', zero_division=0)
f1 = f1_score(y_test, y_pred, average='macro', zero_division=0)

print('Accuracy:', round(accuracy, 4))
print('Precision:', round(precision, 4))
print('Recall:', round(recall, 4))
print('F1-score:', round(f1, 4))

Accuracy: 0.2419
Precision: 0.2159
Recall: 0.2419
F1-score: 0.2157


### Neural Network-based Deep Learning Approach

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
class TransformDataset(Dataset):
    def __init__(self, npz_path, transform=None):
        data = np.load(npz_path)
        self.X = data['X']  
        self.y = data['y']
        self.transform = transform

    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        img = torch.tensor(self.X[idx], dtype=torch.float32)
        img = img.permute(2, 0, 1)

        if self.transform:
            img = self.transform(img)

        return img, int(self.y[idx])

In [6]:
train_tf = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.Normalize(mean=[0.5], std=[0.5]),
])

test_tf = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.Normalize(mean=[0.5], std=[0.5]),
])

In [7]:
train_dataset = TransformDataset('./data/processed/train.npz', transform=train_tf)
val_dataset   = TransformDataset('./data/processed/val.npz',   transform=test_tf)
test_dataset  = TransformDataset('./data/processed/test.npz',  transform=test_tf)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=0)
val_loader   = DataLoader(val_dataset, batch_size=128, shuffle=False, num_workers=0)
test_loader  = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=0)

num_classes = len(np.load('./data/processed/train.npz')['class_names'])

In [8]:
model = efficientnet_b0(weights=EfficientNet_B0_Weights.IMAGENET1K_V1)

# Replace first conv layer so it accepts (1-channel) grayscale
model.features[0][0] = nn.Conv2d(
    1, 32, kernel_size=3, stride=2, padding=1, bias=False
)

# Replace classifier for your classes
model.classifier[1] = nn.Linear(1280, num_classes)

model = model.to(device)

In [9]:
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=20)

In [10]:
def train_one_epoch(model, loader, optimizer, criterion):
    model.train()
    total_loss = 0

    for imgs, labels in loader:
        imgs, labels = imgs.to(device), labels.to(device)

        optimizer.zero_grad()
        out = model(imgs)
        loss = criterion(out, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * imgs.size(0)

    return total_loss / len(loader.dataset)

In [11]:
def evaluate(model, loader):
    model.eval()
    y_true, y_pred = [], []

    with torch.no_grad():
        for imgs, labels in loader:
            imgs = imgs.to(device)
            out = model(imgs)
            preds = torch.argmax(out, dim=1).cpu().numpy()

            y_true.extend(labels.numpy())
            y_pred.extend(preds)

    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='macro', zero_division=0)
    recall = recall_score(y_true, y_pred, average='macro', zero_division=0)
    f1 = f1_score(y_true, y_pred, average='macro', zero_division=0)

    return accuracy, precision, recall, f1

In [ ]:
num_epochs = 40
best_val = 0
patience = 5
patience_counter = 0

for epoch in range(num_epochs):
    train_loss = train_one_epoch(model, train_loader, optimizer, criterion)
    val_acc, val_prec, val_rec, val_f1 = evaluate(model, val_loader)

    print(f'Epoch {epoch+1}/{num_epochs} | Loss {train_loss:.4f} | Val Acc {val_acc:.4f}')

    scheduler.step()

    # Early stopping
    if val_acc > best_val:
        best_val = val_acc
        patience_counter = 0
        torch.save(model.state_dict(), 'best_effnet.pth')
    else:
        patience_counter += 1

    if patience_counter >= patience:
        print('Stopped early')
        break

Epoch 1/40 | Loss 2.6309 | Val Acc 0.6396
Epoch 2/40 | Loss 2.1867 | Val Acc 0.6663
Epoch 3/40 | Loss 2.0686 | Val Acc 0.6803
Epoch 4/40 | Loss 1.9827 | Val Acc 0.6881
Epoch 5/40 | Loss 1.9132 | Val Acc 0.6918
Epoch 6/40 | Loss 1.8469 | Val Acc 0.6992
Epoch 7/40 | Loss 1.7832 | Val Acc 0.7027
Epoch 8/40 | Loss 1.7227 | Val Acc 0.7035
Epoch 9/40 | Loss 1.6562 | Val Acc 0.7074
Epoch 10/40 | Loss 1.5915 | Val Acc 0.7068
Epoch 11/40 | Loss 1.5291 | Val Acc 0.7095
Epoch 12/40 | Loss 1.4672 | Val Acc 0.7089
Epoch 13/40 | Loss 1.4091 | Val Acc 0.7079
Epoch 14/40 | Loss 1.3562 | Val Acc 0.7087
Epoch 15/40 | Loss 1.3106 | Val Acc 0.7081


In [12]:
model.load_state_dict(torch.load('best_effnet.pth'))
model.to(device)

accuracy, precision, recall, f1 = evaluate(model, test_loader)

print('Accuracy:', round(accuracy, 4))
print('Precision:', round(precision, 4))
print('Recall:', round(recall,4))
print('F1:', round(f1, 4))

Accuracy: 0.7096
Precision: 0.7123
Recall: 0.7096
F1: 0.7078


In [13]:
load_dotenv()
login(token=os.getenv('HUGGINGFACE_TOKEN'))

repo_id = 'moosejuice13/cnn_doodle_id_effnet'

api = HfApi()
api.create_repo(repo_id, repo_type='model', exist_ok=True)

api.upload_file(
    path_or_fileobj='best_effnet.pth',
    path_in_repo='cnn_doodle_id_effnet.pth',
    repo_id=repo_id,
)

print(f'Model pushed to https://huggingface.co/{repo_id}')


Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

Model pushed to https://huggingface.co/moosejuice13/cnn_doodle_id_effnet
